# LAB 1 


@author: Abillelatus (Ryan Herrin)

Code for testing and modeling Powershell Obscuration classification using a provided 
data set with predetermined features. PCA may be the preferred method for the unsupervised approach as 
the data provided is not labled. 

Current Models do not include the "sha1" and "fpath" columns


In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns; sns.set(style='white')

%matplotlib inline

In [3]:
# Define Global variables
# Define Global

#original_data_path = "../dataset/all.csv" # this file is originally zipped
original_data_path = "../dataset/all_with_labels.csv" # With Labels 

# Some columns are empty for future use. Set to False to not include them
include_unused_columns = False 

# Define unused columns here
unused_columns = ['vt_harmless', 'vt_undetected', 'vt_malicious', 'vt_suspicious',
				  'avclass_name', 'obf_name']

# Remove the string columns sha1 and fpath
include_sha1_fpath = False

# Remove them from the list if set to False 
if include_sha1_fpath == False:
	unused_columns.append('sha1')
	unused_columns.append('fpath')

+ Create a function that imports the dataset and returns a workable formatted version

In [4]:
def create_workable_data(file_path, lst_unused_columns):
	'''import the dataset and format the data into a pandas dataframe.
	+ Read in CSV as dataframe
	+ Remove extra header if there is one 
	+ Remove unused columns if set 
	+ Typecast DataTypes to string and integers
	+ Returns Pandas DataFrame object'''
	try:
		pd_df = pd.read_csv(file_path) # Read in CSV as dataframe
	except FileNotFoundError():
		print("Could not find data file. Make sure the file is unzipped...")
		sys.exit(1)
		
	# Remove Duplicates
	pd_df.drop_duplicates('sha1', inplace=True)
		
	# Check to see if duplicate header is present. Remove if there is 
	if pd_df.at[0, 'sha1'] == 'sha1':
		pd_df = pd_df.iloc[1:] # .iloc[] integer-loc based indexing for selecting by position 
		
	# Remove unused columns if global var "include_unused_columns" is set to False
	if include_unused_columns == False:
		pd_df = pd_df.drop(columns=lst_unused_columns)

	# Convert all to float 
	for col in pd_df.columns.tolist():
		pd_df[col] = pd_df[col].astype(float)
	
	return(pd_df)

Create reusable function to remove rows from dataframe that are equal to or lower than the value provided. Initially used to remove scripts that
have no length to them. 

In [5]:
def remove_df_rows(data_as_df, char_limit):
    '''Function to remove rows based on the char_limit. char_limit is depicted by
    the doc_char_count feature. 
    @Params:
    - data_as_df (Pandas DataFrame): Data set
    - char_limit (int): Will remove all rows less than or equal to this value
    '''
    working_data = data_as_df # Copy to return
    indx_to_remove = [] # List of index values to remove from DF
    # Find Index's of rows that match the use case 
    for indx in working_data.index.values:
        if float(working_data.at[indx, 'doc_char_count']) <= char_limit:
            indx_to_remove.append(indx)
        else:
            pass

    print('{} row(s) removed that had a document char lenth of {} or less.'.format(
        len(indx_to_remove), char_limit
    ))

    return(working_data.drop(indx_to_remove))

Create Initial dataframes to work with!

In [6]:
data_df = create_workable_data(original_data_path, unused_columns) # Create a Data Frame from the csv 
data_df = remove_df_rows(data_df, 0) # Remove rest of the empty rows 

'''
Note: data_df will be the original data and should not be modified as it will be used
    to compare to delta DataFrames.
'''
# Create Delta Data Frame. This will be used for modifications
delta_data_df = data_df.copy()

1 row(s) removed that had a document char lenth of 0 or less.


Creating more of an investigative function that will show how much of the data is represented depending on a user selected interval. Say you want to know the percentage of script char counts with intervals of 500 to 10,000. It will print out the percentage of 1-500, 501-1000,... 10000-above.

In [6]:
# Show distribution in text first because the current data does not translate over to histogram  
def show_freq_list(data, interval, max):
    """Show a list of amount of scripts that fall between the frequency intervals
    from 0 to the user defined interval. The max set the top limit, as in anything 
    above the max value will be included in the '<Max> and Above'.
    """
    max_limit = max
    low_cnt = 0
    high_cnt = interval
    name_list = []
    count_list = []
    percent_list = [] # Percentage of scripts covered
    
    print("Char Count\t|\tNum of Scripts\t|\tPercent of Data")
    print("----------\t|\t--------------\t|\t---------------")

    while low_cnt < max_limit:
        low_cnt = low_cnt + 1
        name_list.append(str(low_cnt) + '-' + str(high_cnt))

        num_of_instances = len(
            data_df[(data_df['doc_char_count'] >= low_cnt) & (data_df['doc_char_count'] <= high_cnt)]
        )

        count_list.append(num_of_instances)
        percent_list.append(num_of_instances / data.shape[0])

        print('{}\t\t\t{}\t\t{}'.format(name_list[-1], count_list[-1], percent_list[-1]))

        # Update values
        low_cnt = low_cnt + interval -1
        high_cnt = high_cnt + interval

    print('above {}\t\t\t{}\t\t{}'.format(
        max_limit, 
        data.shape[0] - sum(count_list), 
        (data.shape[0] - sum(count_list))/data.shape[0])
    )
    #TODO: Create aggregate. Ex. 1-100: %   |   101-max: %   | max-above: %

show_freq_list(delta_data_df, 5000, 50000)

Char Count	|	Num of Scripts	|	Percent of Data
----------	|	--------------	|	---------------
1-5000			153677		0.7522860779322499
5001-10000			24293		0.11892010965341687
10001-15000			10294		0.05039161934599569
15001-20000			4695		0.022983160368122185
20001-25000			2436		0.011924809085568828
25001-30000			1825		0.008933816330526728
30001-35000			1236		0.0060505188956334445
35001-40000			779		0.003813393381633053
40001-45000			629		0.00307910710789113
45001-50000			427		0.002090268259252007
above 50000			3989		0.0195271196397102


## EDA 

### Analysis

In [7]:
# print out the summary of our pandas DataFrame. Ignore the 'is_obf' column. That is the target column. 
data_df.describe()

,is_obf,char_hash_count,char_paren_count,char_brack_count,char_brace_count,char_bkslash_count,char_fwslash_count,char_dollar_count,char_squote_count,char_dquote_count,...,doc_keyword_return_word_count,doc_keyword_static_word_count,doc_keyword_switch_word_count,doc_keyword_throw_word_count,doc_keyword_trap_word_count,doc_keyword_try_word_count,doc_keyword_until_word_count,doc_keyword_using_word_count,doc_keyword_var_word_count,doc_keyword_while_word_count
count,204280.000000,2.042800e+05,204280.000000,2.042800e+05,204280.000000,204280.000000,2.042800e+05,204280.000000,204280.000000,2.042800e+05,...,204280.000000,204280.000000,204280.000000,204280.000000,204280.000000,204280.000000,204280.000000,204280.000000,204280.000000,204280.000000
mean,2.019703,6.025353e+01,74.440043,5.056465e+01,94.351963,11.562571,7.086974e+01,112.866277,30.245565,6.672977e+01,...,1.277697,0.077521,0.206599,0.432113,0.016051,0.526997,0.041428,0.272983,0.256893,0.153363
std,0.224827,3.665022e+03,576.111577,4.289082e+03,3400.263950,630.926984,3.748793e+03,1789.083560,291.065866,5.685146e+03,...,8.710415,2.402772,1.489893,6.803669,0.278518,6.689347,0.461789,2.055392,2.049805,0.928370
min,1.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000e+00,6.000000,0.000000e+00,4.000000,0.000000,0.000000e+00,11.000000,0.000000,4.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,4.000000e+00,18.000000,6.000000e+00,12.000000,1.000000,2.000000e+00,27.000000,4.000000,1.400000e+01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.000000,1.700000e+01,52.000000,2.400000e+01,34.000000,7.000000,8.000000e+00,70.000000,16.000000,4.000000e+01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.000000,1.280462e+06,165592.000000,1.918417e+06,580658.000000,279010.000000,1.104359e+06,290318.000000,46686.000000,2.564506e+06,...,1837.000000,681.000000,193.000000,2426.000000,51.000000,2427.000000,42.000000,539.000000,191.000000,88.000000


Okay there is a lot of scientific notation present. Huge numbers and small. One way we can try to normalize the data is to make the counts of features a ratio of number of chars are in a documents. This method may help us mitigate cases where very very large scripts can skew the data.

The function below should modify the data frame to transform all features we want transformed excluding the columns that would otherwise have no effect because of num of chars in a file. It can be skipped by setter the 'data_to_percent' variable to False. 

In [8]:
# Option to modify the data to turn feature values from count to ratio
data_to_percent = True

def transform_feature_count_to_percent(dataframe):
    """Function to turn certain features from count to percentage (ratio) values. It
    does this be deviding the feature value by the document char count. Document char count
    is also a feature within the dataset and will be ignored along with some other features.
    This transformation will help try to mitigate the affects of scaling, because as the script 
    gets larger than then char count would go up too. 

    Returns transformed Pandas DataFrame 
    """
    working_data = dataframe  # Create DF to return
    char_count_col_name = 'doc_char_count'

    # List of features that would not likely be susceptible to scaling issues
    non_sus_feat_lst = ['doc_char_count', 'doc_avg_line_len', 'doc_min_line_len',
                        'doc_line_count', 'doc_mcomment_count', 'doc_entropy', 'is_obf']

    # Start looping through the working data and change the values 
    for row in list(working_data.index.values):
        for col_name in working_data.columns.tolist():
            # Skip columns that we don't want to modify
            if col_name not in non_sus_feat_lst:
                working_data.at[row, col_name] = (
                    int(working_data.at[row, col_name]) / int(working_data.at[row, char_count_col_name])
                    )
            else:
                pass

    return(working_data)

# Transform if data_to_percent is true
if data_to_percent:
    delta_data_df = transform_feature_count_to_percent(delta_data_df)

Okay now let's check if it made a difference. Print out the Description of the DataFrame if the transformation was performed. 

In [9]:
# Only runs if the transformation was performed
if data_to_percent:
    delta_data_df.describe()

## PCA

A principle component analysis will be used for this data. We are treating it as an unsupervised approach. Lables were added later to the data to hopefully discover some clustering. We will also use original data along side the delta data for comparison. 

In [10]:
# Lets create a copy of and drop that target column so it doesn't intefere with the PCA 
target_df = pd.DataFrame(data_df['is_obf']) # They are both the same so I only need one. 
# Drop column from DF's 
data_df = data_df.drop(columns=['is_obf'])
delta_data_df = delta_data_df.drop(columns=['is_obf'])

Creating a simple function to determin the best n components for PCA 

In [11]:
# Creating a function to find the optimal n_components for PCA 
def find_opt_n_components(data_set):
	'''Find the number of components that can explain the most data.'''
	set_percentage = .98 # Percentage of data explained

	# The range is the number of components to test
	for comp in range(2, data_set.shape[1]):
		pca = PCA(n_components = comp, random_state=42)
		pca.fit(data_set)
		comp_check = pca.explained_variance_ratio_
		final_comp = comp
		
		if comp_check.sum() >= set_percentage:
			break

	return(final_comp)

opt_n_delta_data = find_opt_n_components(delta_data_df)
print("Optimal n_components for the delta data is {}".format(opt_n_delta_data))

opt_n_data = find_opt_n_components(data_df)
print("Optimal n_components for the original data is {}".format(opt_n_data))

Optimal n_components for the delta data is 2
Optimal n_components for the original data is 4


Now we can create some PCA with our n_components. We will make one for the delta and the original. 

In [13]:
# PCA for the delta 
delta_pca = PCA(n_components=opt_n_delta_data)
x_delta_pca = delta_pca.fit(delta_data_df).transform(delta_data_df) # Fit and transform
print("\n[Delta] Explained variation per principal component: {}\n".format(
	delta_pca.explained_variance_ratio_))

# PCA for the Original Data
orig_pca = PCA(n_components=opt_n_data)
x_orig_pca = orig_pca.fit(data_df).transform(data_df) # Fit and transform
print("\n[Original] Explained variation per principal component: {}\n".format(
	orig_pca.explained_variance_ratio_))


[Delta] Explained variation per principal component: [0.97380168 0.02611931]


[Original] Explained variation per principal component: [0.82619755 0.10880906 0.04209118 0.00652515]

